In [31]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import MDAnalysis as mda
from MDAnalysis.analysis import lineardensity, rms
import nglview as nv

import glob
import subprocess

COMANDO_A_LANZAR_APBS = "/Applications/PyMOL.app/Contents/bin/apbs"

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [24]:
# Carga las dinamicas y selecciona las proteinas
u = mda.Universe("../data/raw/Prot_Lig_ions.prmtop", "../data/raw/TY_DDAT_ions.nc")

# Se quiere seleccionar la proteina, y exportar sus coordenadas para cada frame de la dinamica
protein = u.select_atoms("protein")

In [43]:
# Guarda los archivos en formato PQR
for ts in u.trajectory[::10000]:
    protein.write(f"../data/interim/frame_{ts.frame}.pdb")
    

/Users/javierrojas/anaconda3/envs/apbs-analysis/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/javierrojas/anaconda3/envs/apbs-analysis/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/javierrojas/anaconda3/envs/apbs-analysis/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/javierrojas/anaconda3/envs/apbs-analysis/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/Us

# Calcular APBS para cada frame

In [103]:
import glob
import subprocess

In [112]:
RUTA_ARCHIVOS = "/Users/javierrojas/Documents/Personal/apbs-analysis/data/interim"
RUTA_PYMOL = "/Applications/PyMOL.app/Contents/bin/pymol -c"

In [113]:
ARCHIVOS_DINAMICA = glob.glob(f"{RUTA_ARCHIVOS}/*.pdb")
ARCHIVOS_DINAMICA = sorted(ARCHIVOS_DINAMICA)

In [114]:
for i, ruta_archivo in enumerate(ARCHIVOS_DINAMICA):
    # Extrae el nombre del archivo y el numero del frame a guardar
    nombre_archivo = ruta_archivo.split("/")[-1].split(".")[0]
    numero_archivo = nombre_archivo.split("_")[-1]

    # Modifica el script a correr
    script_a_correr = f"""from pmg_tk.startup.apbs_gui.creating import pdb2pqr_cli
from pmg_tk.startup.apbs_gui.electrostatics import map_new_apbs

cmd.load("{ruta_archivo}")

# Convierte a pqr
pdb2pqr_cli("prepared{numero_archivo}", "{nombre_archivo}", options=["--ff", "amber"])
# Calcula APBS
map_new_apbs("apbs_map{numero_archivo}", "prepared{numero_archivo}")

# Guarda mapa
cmd.save("{RUTA_ARCHIVOS}/prepared{numero_archivo}")
"""

    # Guarda el script para calcular APBS de frame
    with open(f"../data/external/script.py", "w") as file:
        file.write(script_a_correr)
        
    
    # Escribe el comando a correr y lo corre en la consola
    COMANDO_A_CORRER = f"{RUTA_PYMOL} ../data/external/script.py"
    process = subprocess.Popen(
        COMANDO_A_CORRER, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    
    if i == 0:
        break